In [41]:
import pandas as pd
import numpy as np

In [42]:
data = pd.read_csv("./data/gemstone.csv")

In [43]:
data.drop('id',axis =1,inplace = True)

In [44]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [45]:
X = data.drop('price',axis = 1 )

In [46]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [47]:
y = data['price']





In [48]:
y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [49]:
cat_column = X.select_dtypes(include='object').columns

In [50]:
cat_column

Index(['cut', 'color', 'clarity'], dtype='object')

In [51]:
num_column = X.select_dtypes(exclude='object').columns

In [52]:
num_column

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [53]:
from sklearn.impute import SimpleImputer # for handling missing value
from sklearn.preprocessing import StandardScaler # Handling feature scaling
from sklearn.preprocessing import OrdinalEncoder # ordinal encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [54]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [55]:
numerical_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer()),
        ('scalar',StandardScaler())
    ]
)

In [56]:
categorical_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoding',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
)

In [57]:
preprocessor = ColumnTransformer(
    [
        ('num_pipeline',numerical_pipeline,num_column),
        ('cat_pipline',categorical_pipeline,cat_column)

    ]
)

In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=40)

In [59]:
preprocessor.fit_transform(X_train)

array([[ 0.45410567, -0.29654463, -0.63784219, ...,  4.        ,
         2.        ,  2.        ],
       [ 3.02766057, -1.58999042,  1.96548435, ...,  3.        ,
         4.        ,  1.        ],
       [ 0.54061172,  1.45884607, -1.15850749, ...,  2.        ,
         4.        ,  1.        ],
       ...,
       [ 2.61675685,  1.82840201, -0.63784219, ...,  2.        ,
         4.        ,  1.        ],
       [ 0.45410567,  0.99690115, -1.6791728 , ...,  1.        ,
         3.        ,  4.        ],
       [-0.88673805, -1.12804549,  2.48614965, ...,  2.        ,
         1.        ,  5.        ]])

In [60]:
preprocessor.transform(X_test)

array([[ 3.33043173,  0.16540029, -0.63784219, ...,  4.        ,
         4.        ,  1.        ],
       [-0.36770177,  0.16540029, -0.11717688, ...,  4.        ,
         2.        ,  2.        ],
       [ 0.8650094 , -0.20415565, -1.15850749, ...,  4.        ,
         4.        ,  2.        ],
       ...,
       [-0.71372596, -0.85087854, -0.63784219, ...,  4.        ,
         1.        ,  4.        ],
       [ 1.12452754,  0.81212318,  0.40348843, ...,  3.        ,
         4.        ,  1.        ],
       [-0.9732441 , -0.75848956,  0.40348843, ...,  3.        ,
         1.        ,  3.        ]])

In [61]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipline__cut', 'cat_pipline__color',
       'cat_pipline__clarity'], dtype=object)

In [63]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [73]:
dummy = np.arange(20).reshape(5,4)
dummy

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [76]:
dummy[:,-1]

array([ 3,  7, 11, 15, 19])

In [ ]:
X_train_arr = np.c_[X_train,np.array(y_train)]
X_train_arr.shape

(135501, 10)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
import numpy as np
def evaluate_model(true,predict):
    mae = mean_absolute_error(true,predict)
    mse = mean_squared_error(true,predict)
    rmse = np.sqrt(mean_squared_error(true,predict))
    r2_square = r2_score(true,predict)
    return mae ,rmse,mse,r2_square

In [ ]:
#train mutiple model
models = {
    'Linear':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

trained_model = []
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    # make prediction
    y_pred = model.predict(X_test)
    MAE , RMSE,mse , r2_sqaured = evaluate_model(y_test,y_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print('RMSE',RMSE)
    print("MEA",MAE)
    print("R2 score",r2_sqaured*100)

    r2_list.append(r2_sqaured)

    print('='*35)
    print('\n')

Linear
Model Training Performance
RMSE 1017.1722564866152
MEA 674.6638217377888
R2 score 93.63431403063537


Lasso
Model Training Performance
RMSE 1015.5985590975571
MEA 675.5232012328668
R2 score 93.65399587707492


Ridge
Model Training Performance
RMSE 1017.1731060578769
MEA 674.6877442317139
R2 score 93.6343033970262


Elasticnet
Model Training Performance
RMSE 1517.159490868822
MEA 1055.223208205003
R2 score 85.83819290551544


